In [1]:
import pandas as pd
import numpy as np

In [2]:
cl = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Parametriche\Classificazione_codice_prodotto.xlsx"
match = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Analisi clienti\clienti_insides_v2.xlsx"

In [4]:
df_cl = pd.read_excel(cl, engine="openpyxl")

In [5]:
df_match = pd.read_excel(match, sheet_name="BW", engine="openpyxl", dtype={"Cliente MERCE":"string"})
df_bridio = pd.read_excel(match,sheet_name="bridio", engine="openpyxl", dtype={"COD.CLI":"string", "CAP":"string","COD.FISCALE":"string","PART.IVA":"string"})

In [9]:
filt = (df_match["Consenso Privacy"] == "Senza attribuz.") & (df_match["No TVA"] == "Senza attribuz.") & (df_match["Sales Director"].str.contains("ARTEL DIRETTA|ENTERPRISE|LARGE ACCOUNT|Senza attribuz.|WKIC CENTRO SUD|WKIC NORD EST|WKIC NORD OVEST"))
columns_match = ["Cliente MERCE", "Regione", "Tipo Partner Aggrega", "Sales Director", "Cod Agente dell'Abbonamento", "Agente dell'Abbonamento", "Prodotto (SW)", "Soluzione (SW)", "Cluster (SW)", "Area (SW)", "Canale di VENDITA", "Valore PV New"]
columns_bridio =["COD.CLI", "RAG.SOC.1", "RAG.SOC.2", "RAG.SOC.3", "INDIRIZZO", "LOCALITA'", "TELEFONO 1", "TELEFONO 2", "PART.IVA"]

pv_target = df_match.loc[filt].groupby(["Cliente MERCE"]).agg({"Valore PV New":"sum"}).reset_index()
pv_target = pv_target.loc[(pv_target["Valore PV New"] > 0) & (pv_target["Valore PV New"] < 1200), :]

df = df_match.loc[filt, columns_match].merge(pv_target.loc[:,"Cliente MERCE"], on="Cliente MERCE", how="inner").merge(df_bridio.loc[:, columns_bridio], left_on="Cliente MERCE", right_on="COD.CLI", how="left").merge(df_cl.loc[:,["Prodotto (SW)","Business area"]].drop_duplicates(subset="Prodotto (SW)"), on="Prodotto (SW)", how="left")


In [11]:
df.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Analisi clienti\clienti_insides_v4.xlsx", index=False)

In [ ]:
df_gr = df.groupby(["Cliente MERCE","PART.IVA","RAG.SOC.1","RAG.SOC.2","RAG.SOC.3","TELEFONO 1","TELEFONO 2","Regione","Sales Director","Cod Agente dell'Abbonamento","Agente dell'Abbonamento","Business area"], dropna=False).agg({"Valore PV New":"sum"}).reset_index()

In [ ]:
df_gr.columns = ["Cliente merce",  "PIVA" , "Ragione Sociale 1", "Ragione Sociale 2", "Ragione Sociale 3", "Telefono 1", "Telefono 2", "Regione", "Area cliente", "Cod. Sales" , "Rag. Soc. Sales", "Area PV", "PV"]

In [ ]:
df_gr.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Reports\Analisi clienti\clienti_insides_v3.xlsx", index=False)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5747 entries, 0 to 5746
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Cliente MERCE                5747 non-null   object 
 1   Regione                      5747 non-null   object 
 2   Tipo Partner Aggrega         5747 non-null   object 
 3   Sales Director               5747 non-null   object 
 4   Cod Agente dell'Abbonamento  5747 non-null   object 
 5   Agente dell'Abbonamento      5747 non-null   object 
 6   Prodotto (SW)                5747 non-null   object 
 7   Soluzione (SW)               5747 non-null   object 
 8   Cluster (SW)                 5747 non-null   object 
 9   Area (SW)                    5747 non-null   object 
 10  Canale di VENDITA            5747 non-null   object 
 11  Valore PV New                5747 non-null   float64
 12  COD.CLI                      5708 non-null   string 
 13  RAG.SOC.1         